In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import tensorflow as tf
import pandas as pd

In [2]:
labels = pd.read_csv("https://raw.githubusercontent.com/nightrome/cocostuff/master/labels.txt")
labels = pd.DataFrame(labels["0: unlabeled"].str.split(':',1).tolist(),
                                   columns = ['label','name'])
labels["label"] = labels["label"].astype(int)
labels = labels.set_index("label")
labels_dict = labels.to_dict()
list_images = []

In [3]:
import urllib.request
import os
import tarfile
import shutil

base_url = 'http://download.tensorflow.org/models/object_detection/'
file_name = 'ssd_mobilenet_v1_coco_2018_01_28.tar.gz'
url = base_url + file_name
urllib.request.urlretrieve(url, file_name)
os.listdir()

dir_name = file_name[0:-len('.tar.gz')]

if os.path.exists(dir_name):
  shutil.rmtree(dir_name) 

tarfile.open(file_name, 'r:gz').extractall('./')
os.listdir(dir_name)

['checkpoint',
 'frozen_inference_graph.pb',
 'model.ckpt.data-00000-of-00001',
 'model.ckpt.index',
 'model.ckpt.meta',
 'pipeline.config',
 'saved_model']

In [4]:
import tensorflow as tf


#Open the frozen_graph file
#Load the graph into memory
with tf.Session() as sess:
    frozen_graph = os.path.join(dir_name, 'frozen_inference_graph.pb')
    with tf.gfile.FastGFile(frozen_graph,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='/logs/tests/1/'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)
    
print("Model Imported. Visualize by running: "
          "tensorboard --logdir={}".format(LOGDIR))

train_writer.flush()
train_writer.close()


Model Imported. Visualize by running: tensorboard --logdir=/logs/tests/1/


In [5]:
sess.close()

In [6]:
outputs = (
    'num_detections',
    'detection_classes',
    'detection_scores',
    'detection_boxes',
)

In [7]:
detections =[]

In [8]:
cap = cv.VideoCapture(0)

In [ ]:
with tf.Session() as sess:
    sess.graph.as_default()
    tf.import_graph_def(graph_def,name='')
    while True:
        ret, input_images = cap.read()
        image_np_expanded = np.expand_dims(input_images, axis=0)

        detections = (sess.run(
            [sess.graph.get_tensor_by_name(f'{op}:0') for op in outputs],
            feed_dict={ 'image_tensor:0': image_np_expanded }

          ))
        num_objects = int(detections[0][0])
        for j in range(num_objects):
            if detections[2][0][j]>0.5:
                l_y = int(detections[3][0][j][0]*360)
                l_x = int(detections[3][0][j][1]*640)
                r_y = int(detections[3][0][j][2]*360)
                r_x = int(detections[3][0][j][3]*640)
                #pct = detections[0][2][0][i]*100
                #pct = round(pct,2)
                #plt.imshow(cv.rectangle(list_images[i],(l_x,l_y),(r_x,r_y),color = (255,0,0),thickness = 10))

                cv.rectangle(input_images,(l_x,l_y),(r_x,r_y),color = (255,0,0),thickness = 5)
                cv.putText(input_images, str(labels_dict["name"][detections[1][0][j]]), (l_x, l_y), cv.FONT_HERSHEY_SIMPLEX, 1.2, [255,0,0], thickness = 2)
                cv.putText(input_images, str(detections[2][0][j]), (l_x, r_y), cv.FONT_HERSHEY_SIMPLEX, 0.8, [0,255,0], thickness = 2)
        cv.imshow("img",cv.resize(input_images, (1000,800)))
        if (cv.waitKey(25) & 0xFF == ord('q')):
            cv.destroyAllWindows()
            break